In [40]:
import os
os.environ["JAX_PLATFORM_NAME"] = "cpu"
import netket as nk
import numpy as np
import matplotlib.pyplot as plt
import jax

In [41]:
import warnings
warnings.filterwarnings("ignore")

In [42]:
#Couplings J1 and J2
J = [1, 0.2]
L = 8

In [43]:
# Define custom graph
edge_colors = []
for i in range(L):
    edge_colors.append([i, (i+1)%L, 1])
    edge_colors.append([i, (i+2)%L, 2])
# Define the netket graph object
g = nk.graph.Graph(edges=edge_colors)

In [44]:
#Sigma^z*Sigma^z interactions
sigmaz = [[1, 0], [0, -1]]
mszsz = (np.kron(sigmaz, sigmaz))
exchange = np.asarray([[0, 0, 0, 0], [0, 0, 2, 0], [0, 2, 0, 0], [0, 0, 0, 0]])

bond_operator = [
    (J[0] * mszsz).tolist(),
    (J[1] * mszsz).tolist(),
    (-J[0] * exchange).tolist(),  
    (J[1] * exchange).tolist(),
]

bond_color = [1, 2, 1, 2]

In [45]:
hi = nk.hilbert.Spin(s=0.5, total_sz=0.0, N=g.n_nodes)

In [46]:
op = nk.operator.GraphOperator(hi, graph=g, bond_ops=bond_operator, bond_ops_colors=bond_color)

In [47]:
import netket.nn as nknn
import flax.linen as nn
import jax.numpy as jnp

class FFNN(nn.Module):
    @nn.compact
    def __call__(self, x):
        x = nn.Dense(features=2*x.shape[-1], 
                     use_bias=True, 
                     param_dtype=np.complex128, 
                     kernel_init=nn.initializers.normal(stddev=0.01), 
                     bias_init=nn.initializers.normal(stddev=0.01)
                    )(x)
        x = nknn.log_cosh(x)
        x = jnp.sum(x, axis=-1)
        return x

model = FFNN()

In [48]:
sa = nk.sampler.MetropolisExchange(hilbert=hi, graph=g, d_max = 2)
vs = nk.vqs.MCState(sa, model, n_samples=1008)
opt = nk.optimizer.Sgd(learning_rate=0.01)
sr = nk.optimizer.SR(diag_shift=0.01)
gs = nk.VMC(hamiltonian=op, optimizer=opt, variational_state=vs, preconditioner=sr)

In [49]:
vs

MCState(
  hilbert = Spin(s=1/2, total_sz=0.0, N=8),
  sampler = MetropolisSampler(rule = ExchangeRule(# of clusters: 28), n_chains = 16, n_sweeps = 8, reset_chains = False, machine_power = 2, dtype = <class 'float'>),
  n_samples = 1008,
  n_discard_per_chain = 100,
  sampler_state = MetropolisSamplerState(rng state=[2647336765 3615970261]),
  n_parameters = 144)

In [62]:
vs_i_parameters['Dense_0']['bias']

Array([ 0.00251272-0.00988675j, -0.00635292-0.00547227j,
        0.00392519+0.00064911j,  0.01189758+0.00403939j,
        0.00925428+0.00159196j,  0.00134483-0.01311992j,
       -0.00311009+0.0019083j , -0.01207187+0.00580082j,
        0.00176632+0.00842852j, -0.00708888+0.00527055j,
        0.02008881+0.00174007j, -0.0064344 +0.01264528j,
       -0.01143401+0.00442696j,  0.0051914 +0.00510249j,
       -0.00361964-0.00530344j,  0.00246203+0.00114288j],      dtype=complex128)

In [55]:
# Define a função de perda (loss function) para o treinamento
def loss(params, structure_factor):
    output = model.apply({'params': params}, structure_factor)
    return jnp.mean(output)

In [56]:
sf = []
sites = []
structure_factor = nk.operator.LocalOperator(hi, dtype=complex)
for i in range(0, L):
    for j in range(0, L):
        structure_factor += (nk.operator.spin.sigmaz(hi, i)*nk.operator.spin.sigmaz(hi, j))*((-1)**(i-j))/L

In [58]:
# Define uma lista para armazenar os parâmetros do modelo durante o treinamento
trained_params_list = []
parameters_list     = []
iii                 = []
# Função de callback para salvar os parâmetros durante o treinamento
def save_params(step, params, energy):
    #print("############ it    ", 1 + len(iii))
    #print("############ energy", energy.state.parameters)
    
    trained_params_list.append(params.copy())
    parameters_list.append(energy.state.parameters.copy())
    iii.append(1)
    return True

In [59]:
gs.run(out='test', n_iter=300, obs={'Structure Factor': structure_factor}, callback=save_params)

100%|████████████████████████████████████████████████████████████████████████████| 300/300 [00:19<00:00, 15.41it/s, Energy=-13.40274+0.00022j ± 0.00044 [σ²=0.00020, R̂=1.0093]]


(JsonLog('test', mode=write, autoflush_cost=0.005)
   Runtime cost:
   	Log:    0.06972908973693848
   	Params: 0.001984119415283203,)

In [63]:
vs_i_parameters['Dense_0']['bias']

Array([ 0.00251272-0.00988675j, -0.00635292-0.00547227j,
        0.00392519+0.00064911j,  0.01189758+0.00403939j,
        0.00925428+0.00159196j,  0.00134483-0.01311992j,
       -0.00311009+0.0019083j , -0.01207187+0.00580082j,
        0.00176632+0.00842852j, -0.00708888+0.00527055j,
        0.02008881+0.00174007j, -0.0064344 +0.01264528j,
       -0.01143401+0.00442696j,  0.0051914 +0.00510249j,
       -0.00361964-0.00530344j,  0.00246203+0.00114288j],      dtype=complex128)

In [60]:
vs.parameters['Dense_0']['bias']

Array([ 0.00180268-0.00252496j,  0.00136733-0.00056385j,
        0.00212432-0.00436502j,  0.00554233+0.00246408j,
       -0.00117289+0.00021168j,  0.00347624-0.00024031j,
       -0.00221386+0.00177004j, -0.00255286+0.00114417j,
       -0.00114083+0.00243313j, -0.00512626+0.0052904j ,
        0.00383401+0.00088031j,  0.00084268+0.00596297j,
       -0.00206288+0.00124847j,  0.00049361+0.00082766j,
       -0.00140611+0.00111019j,  0.00150738-0.00154616j],      dtype=complex128)

In [64]:
parameters_list[-1]['Dense_0']['bias']

Array([ 0.00180217-0.00251152j,  0.00135909-0.00057767j,
        0.00211597-0.00435713j,  0.00555341+0.00246417j,
       -0.00116199+0.00021222j,  0.00349   -0.00026497j,
       -0.00221237+0.00178953j, -0.00255608+0.00114619j,
       -0.00114757+0.00244537j, -0.00511356+0.00526814j,
        0.00385049+0.00088343j,  0.00084044+0.00597642j,
       -0.00207499+0.00127593j,  0.00049015+0.00081769j,
       -0.00142138+0.00109876j,  0.00150515-0.00154129j],      dtype=complex128)

In [ ]:
list(vs_i_parameters['Dense_0']['kernel']

In [38]:
list(vs.parameters['Dense_0']['kernel'])[0]

Array([ 0.00240868-0.05934615j,  0.14920958+0.08744754j,
       -0.12718667+0.10624626j, -0.00565899+0.19991692j,
        0.00744991+0.12459088j,  0.09170598-0.0074441j ,
        0.05933686-0.05879388j, -0.06972199+0.21521495j,
        0.09352375+0.23475587j,  0.07308585-0.07340458j,
       -0.09164095-0.04256573j, -0.03415678+0.21244502j,
       -0.15320064-0.00337424j, -0.01880855+0.18705987j,
       -0.12531965-0.11590249j, -0.00340865-0.01036198j],      dtype=complex128)

In [39]:
list(parameters_list[-1]['Dense_0']['kernel'])[0]

Array([ 0.00244307-0.05930074j,  0.149243  +0.08746194j,
       -0.12718226+0.10622449j, -0.00566049+0.19998888j,
        0.00740833+0.12457445j,  0.09170855-0.00745465j,
        0.05937607-0.05877456j, -0.0697046 +0.21508718j,
        0.09350699+0.23480969j,  0.07307471-0.07337257j,
       -0.09165947-0.04256373j, -0.03409473+0.21243304j,
       -0.15315985-0.00334263j, -0.01886052+0.1870421j ,
       -0.12526718-0.11589971j, -0.00341532-0.01034179j],      dtype=complex128)

In [ ]:
# Load the data from the .log file
import json

data=json.load(open("test.log"))

iters = data['Energy']['iters']
energy=data['Energy']['Mean']['real']
sf=data['Structure Factor']['Mean']['real']

In [ ]:
fig, ax1 = plt.subplots()
ax1.plot(iters, energy, color='blue', label='Energy')
ax1.set_ylabel('Energy')
ax1.set_xlabel('Iteration')
ax2 = ax1.twinx() 
ax2.plot(iters, np.array(sf), color='green', label='Structure Factor')
ax2.set_ylabel('Structure Factor')
ax1.legend(loc=2)
ax2.legend(loc=1)
plt.show()

In [ ]:
print(r"Structure factor = {0:.3f}({1:.3f})".format(np.mean(sf[-50:]),
                                              np.std(np.array(sf[-50:]))/np.sqrt(50)))
print(r"Energy = {0:.3f}({1:.3f})".format(np.mean(energy[-50:]), np.std(energy[-50:])/(np.sqrt(50))))

In [ ]:
E_gs, ket_gs = nk.exact.lanczos_ed(op, compute_eigenvectors=True)
structure_factor_gs = (ket_gs.T.conj()@structure_factor.to_linear_operator()@ket_gs).real[0,0]

In [ ]:
print("Exact Ground-state Structure Factor: {0:.3f}".format(structure_factor_gs))
print("Exact ground state energy = {0:.3f}".format(E_gs[0]))